In [1]:
import pandas as pd

In [2]:
import csv

In [23]:
data = pd.read_csv('train.csv', index_col=0)

In [28]:
data.iloc[0].question1

'What is the step by step guide to invest in share market in india?'

In [ ]:
# deal with [math]23^{24}[/math] 

# удалить пустые строки из файлика трейн и перезаписать его

# добавить в предобработку -- lemmas
# tf-idf
# w2vec + n_similarity(ws1, ws2)
# vw (vowpal wabbit)
# счетчики
# биграммы

# скрипт должен фититься по трейну и предсказывать на тесте

### Preprocessing & Bag of words

In [3]:
from nltk.corpus import stopwords
import nltk
import re

In [22]:
nltk.pos_tag(['cat', 'ran', 'home'])

[('cat', 'NN'), ('ran', 'VBD'), ('home', 'NN')]

In [12]:
nltk.pos_tag?

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(tagged_token):
    token, tag = tagged_token
    return lemmatizer.lemmatize(token, pos=pos_tranform[tag])

In [21]:
nltk.pos_tag(['ran'])

[('ran', 'NN')]

In [20]:
list(map(lemmatize, nltk.pos_tag(['went', 'cat', 'home'])))

['go', 'cat', 'home']

In [9]:
pos_tranform = {'JJ':'a', 'JJR':'a', 'JJS':'a', 'NN':'n', 'NNS':'n', 'NNP':'n', 'NNPS':'n', 'RB':'r', 'RBR':'r', 
                'RBS':'r', 'VB':'v', 'VBD':'v', 'VBG':'v', 'VBN':'v', 'VBP':'v', 'VBZ':'v'}

In [33]:
pos_tranform = {'JJ':'a', 'JJR':'a', 'JJS':'a', 'NN':'n', 'NNS':'n', 'NNP':'n', 'NNPS':'n', 'RB':'r', 'RBR':'r', 
                'RBS':'r', 'VB':'v', 'VBD':'v', 'VBG':'v', 'VBN':'v', 'VBP':'v', 'VBZ':'v'}

def lemmatize(tagged_token):
    token, tag = tagged_token
    return lemmatizer.lemmatize(token, pos=pos_tranform.get(tag, 'n'))

def preprocessing(text):
#     try:
    # Dealing with Punctuation and numbers
    clean_text = re.sub("[^\w]", ' ', text)

#     lower-case and split into words
    clean_text = clean_text.lower().split()
    
    clean_text = list(map(lemmatize, nltk.pos_tag(clean_text)))
#     stopwords
    words = [word for word in clean_text if word not in stopwords.words('english')]

    return (' '.join(words))
#     except:
#         print(text)
#         return text

In [34]:
s = 'What is the step by step guide to invest in share market in india?'
print(s)
s = re.sub("[^\w]", ' ', s)
print(s)
s = clean_text = s.lower().split()
print(s)
s = list(map(lemmatize, nltk.pos_tag(s)))
print(s)
s = [word for word in s if word not in stopwords.words('english')]
print(s)

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market in india 
['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
['what', 'be', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
['step', 'step', 'guide', 'invest', 'share', 'market', 'india']


In [9]:
# data = data[~data.question2.isnull()]
data = data.dropna(how="any")

In [6]:
data['question1'] = data['question1'].apply(preprocessing)

In [7]:
data['question2'] = data['question2'].apply(preprocessing)

nan
nan


In [8]:
data.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,step step guide invest share market india,step step guide invest share market,0
1,3,4,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,0
2,5,6,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,7,8,mentally lonely solve,find remainder math math divided,0
4,9,10,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0


In [35]:
data.is_duplicate.value_counts(normalize=True)

0    0.6308
1    0.3692
Name: is_duplicate, dtype: float64

### Split data

In [60]:
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
import os

In [50]:
# skf = StratifiedKFold(data['is_duplicate'], n_folds=5, shuffle=True, random_state=42)

In [59]:
# !mkdir -p f{0..4}

In [55]:
# for fold, (train_ids, test_ids) in enumerate(skf):
#     folder = os.path.join('./', 'f{}'.format(fold))
#     data.loc[train_ids].to_csv(os.path.join(folder, 'train.csv'))
#     data.loc[test_ids].to_csv(os.path.join(folder, 'test.csv'))

0.37076108733829677
0.36774799955476956
0.37071161789804347
0.3682055975363914
0.36858118135945384


In [12]:
x_train, x_test, y_train, y_test = train_test_split(data[['question1', 'question2']], data['is_duplicate'], 
                                                    test_size=0.3, stratify=data['is_duplicate'],
                                                    random_state=42)

In [13]:
x_train.head()

,question1,question2
id,,
155515,best books iit jam mathematics,best books iit jam mathematics
400580,best photo startups,good photo
22595,best revenge rejected someone,best revenge
15217,story first camera invented,camera invented
67927,add app vizio smart tv,youtube like apps sharing smart tv


### Cooking Bag of words

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
from sklearn.metrics.pairwise import paired_distances

In [30]:
import numpy as np

In [11]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, 
                             preprocessor = None, stop_words = None, max_features = 5000)

In [14]:
concated = pd.concat([x_train['question1'], x_train['question2']])

In [15]:
x_fit = vectorizer.fit(concated)

In [27]:
x_train1.shape[0]

283001

In [ ]:
def get_cosine_similarities(model, x_train):
    x_train1 = model.transform(x_train['question1']).toarray()
    x_train2 = model.transform(x_train['question2']).toarray()
    
    cosines = []
#     cosines!!!!
    for i in range(6):
        cosines.append(paired_distances(x_train1[50000*i:50000*(i+1)], x_train2[50000*i:50000*(i+1)], metric='cosine'))
    
    return np.concatenate(cosines)

In [16]:
x_train1 = vectorizer.transform(x_train['question1']).toarray()
x_train2 = vectorizer.transform(x_train['question2']).toarray()

In [28]:
cosines = []
for i in range(6):
    cosines.append(paired_distances(x_train1[50000*i:50000*(i+1)], x_train2[50000*i:50000*(i+1)], metric='cosine'))

In [32]:
cosine = np.concatenate(cosines)

In [29]:
cosines

[array([ 0.        ,  0.59175171,  0.29289322, ...,  0.0741799 ,
         0.81742581,  0.45566895]),
 array([ 0.04257289,  0.        ,  1.        , ...,  0.5       ,
         1.        ,  0.5       ]),
 array([ 0.74645372,  0.2       ,  0.26970326, ...,  0.71132487,
         0.42264973,  1.        ]),
 array([ 1.        ,  0.59175171,  1.        , ...,  0.0741799 ,
         0.22848325,  0.        ]),
 array([ 0.18181818,  0.18350342,  0.33333333, ...,  0.1339746 ,
         0.25      ,  0.25      ]),
 array([ 0.42264973,  0.88047714,  0.        , ...,  0.32917961,
         0.59175171,  0.42264973])]

In [33]:
cosine.shape

(283001,)

In [37]:
y_train.shape

(283001,)

### ML

In [34]:
from sklearn.linear_model import LogisticRegressionCV

In [35]:
model = LogisticRegressionCV(n_jobs=4)

In [38]:
model.fit(cosine[:, np.newaxis], y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=4, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

### Quality

In [39]:
from sklearn.metrics import log_loss, roc_auc_score

In [43]:
y_pred = model.predict_proba(cosine[:, np.newaxis])[:, 1]

In [44]:
log_loss(y_train, y_pred)

0.58972109908275994

In [45]:
roc_auc_score(y_train, y_pred)

0.70873730927328527

In [47]:
# константный прогноз
log_loss(y_train, [y_train.mean()] * len(y_train))

0.658528592318252

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
# print tfidf_matrix.shape
# (4, 11)
# Now we have the TF-IDF matrix (tfidf_matrix) for each document (the number of rows of the matrix) with 11 tf-idf terms (the number of columns from the matrix), we can calculate the Cosine Similarity between the first document (“The sky is blue”) with each of the other documents of the set:

# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
# array([[ 1.        ,  0.36651513,  0.52305744,  0.13448867]])